# **Fundamentals of Artificial Intelligence**

## MSc in Applied Artificial Intelligence 2025/2026 <br>
## Group 02 - Project 2 - Decision Tree and Random Forest
| Nome                              | Número de Aluno |
|-----------------------------------|------------:|
| Adelino Daniel da Rocha Vilaça    | a16939          |
| António Jorge Magalhães da Rocha  | a26052          |

---
# 0. - **INTRODUCTION**

## 0.1 - Goal
> O objetivo principal deste projeto é desenvolver e avaliar modelos de *machine learning* (Árvore de Decisão e Random Forest) para a deteção de intrusões em cibersegurança. Pretende-se classificar sessões de rede como normais ou maliciosas (`attack_detected`) utilizando um dado *dataset*, e compreender as características-chave que contribuem para tais deteções.

## 0.2 - Environment
> O projeto é desenvolvido num ambiente Google Colaboratory, aproveitando os seus serviços de Jupyter Notebook baseados na *cloud*. A implementação utiliza a linguagem de programação Python e um conjunto de bibliotecas populares de ciência de dados e *machine learning*, incluindo `pandas` para manipulação de dados, `numpy` para operações numéricas, `matplotlib` e `seaborn` para visualização de dados, e `scikit-learn` para modelos de *machine learning* (Árvore de Decisão, Random Forest) e métricas de avaliação. A API do Kaggle é utilizada para descarregar o *dataset*.

## 0.3 - Definitions
-   **Árvore de Decisão (Decision Tree):** Um algoritmo de aprendizagem supervisionada não-paramétrico usado para tarefas de classificação e regressão. Constrói um modelo em forma de estrutura de árvore, onde os nós internos representam testes numa característica, os ramos representam o resultado do teste e os nós folha representam rótulos de classe ou valores.
-   **Random Forest:** Um método de aprendizagem por *ensemble* para classificação, regressão e outras tarefas que opera construindo uma infinidade de árvores de decisão durante o treino e produzindo a classe que é o modo das classes (classificação) ou a previsão média (regressão) das árvores individuais. Ajuda a reduzir o *overfitting* e a melhorar a precisão.
-   **Deteção de Intrusão em Cibersegurança:** O processo de monitorizar uma rede ou sistema informático para atividades maliciosas ou violações de políticas. Qualquer intrusão detetada é tipicamente reportada a um administrador ou recolhida centralmente usando um sistema de gestão de eventos e informações de segurança (SIEM).
-   **Overfitting:** Um erro de modelagem que ocorre quando uma função se alinha demasiado a um conjunto limitado de pontos de dados. Significa que o modelo aprende os dados de treino e o seu ruído demasiado bem, levando a um desempenho fraco em dados não vistos.
-   **`max_depth`:** Um hiperparâmetro em modelos baseados em árvores que especifica a profundidade máxima da árvore, da raiz à folha. Limitar a profundidade ajuda a prevenir o *overfitting*.
-   **`LabelEncoder`:** Uma utilidade em `scikit-learn` usada para normalizar os rótulos de modo a que contenham apenas valores entre 0 e n_classes-1. É frequentemente usado para codificar características categóricas em formato numérico.

---
# 1. - **AGENT DESIGN**

## 1.1 - Platforms

### 1.1.1 - Jupyter Notebook <br>
 > A Jupyter Notebook is an open-source web application that allows creating and sharing documents containing live code, equations, visualizations, and narrative text. It's widely used in data science, machine learning, and scientific computing for interactive development, exploration, and documentation.

### 1.1.2 - Google Colaboratory <br>
  >Free cloud-based service that provides a hosted Jupyter Notebook environment. It allows writing and executing code in a browser for free and without any setup.

## 1.2 Packages and Libraries


### 1.2.1 Explicar "libs"
- pandas (pd): Essencial para manipulação e análise de dados em formato de tabela (DataFrames).

- numpy (np): A base para operações numéricas e matemáticas com arrays e matrizes.

- matplotlib.pyplot (plt): Usada para criar gráficos e visualizações de dados estáticos.

- seaborn (sns): Construída sobre o Matplotlib, serve para criar gráficos estatísticos mais complexos e esteticamente agradáveis.

- sklearn.model_selection: Contém ferramentas para dividir dados (treino/teste) e realizar validação cruzada.

- sklearn.tree: Oferece algoritmos para Árvores de Decisão e ferramentas para as visualizar.

- sklearn.ensemble: Implementa métodos de conjunto como o Random Forest, que combina múltiplos modelos para melhor desempenho.

- sklearn.metrics: Fornece métricas para avaliar o desempenho dos modelos, como precisão, recall, F1-score e matriz de confusão.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

RANDOM_STATE = 27

### Uso do `kaggle.json`

O ficheiro `kaggle.json` é um **token de autenticação** essencial para interagir com a API do Kaggle (Kaggle Application Programming Interface). Ele contém as credenciais de utilizador do Kaggle (username e key) de forma segura.

In [ ]:
from google.colab import files
files.upload()  # kaggle.json

### Permissões para o Token

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list | head

## 1.3 DATASET


### 1.3.1 Explicação Dataset Usado

> Este dataset, denominado `cybersecurity_intrusion_data.csv`, foca-se na **deteção de intrusões em cibersegurança**.

*   Contém informações sobre sessões de rede, como tamanho dos pacotes, tipo de protocolo, duração da sessão e tentativas de login.
*   Inclui detalhes sobre o comportamento do utilizador, como o tipo de navegador e tentativas falhadas de login.
*   O objetivo principal é classificar se uma determinada sessão indica um `attack_detected` (ataque detetado) ou não, sendo esta a variável alvo.


In [ ]:
!kaggle datasets download -d dnkumars/cybersecurity-intrusion-detection-dataset -p /content/ --unzip

---
# 2. **AGENT RUNNING**

### 2.1 Leitura e Pré-visualização do Dataset

Esta célula é responsável por carregar o dataset `cybersecurity_intrusion_data.csv` para um DataFrame do pandas e realizar uma inspeção inicial.

* **`dataset = pd.read_csv('cybersecurity_intrusion_data.csv')`**: Carrega o ficheiro CSV para uma variável `dataset`.
* **`df = dataset`**: Cria uma cópia do DataFrame `dataset` na variável `df` para facilitar a manipulação.
* **`dataset_data = pd.DataFrame(dataset)`**: Cria outra cópia, `dataset_data`, que será utilizada para algumas visualizações ou transformações que preservam o `session_id`.
* **`print("Shape:", df.shape)`**: Exibe as dimensões do DataFrame (número de linhas e colunas).
* **`dataset_data.head()`**: Mostra as primeiras 5 linhas do DataFrame, permitindo uma rápida visualização da estrutura e dos tipos de dados.

In [ ]:
#0. Read the file
dataset = pd.read_csv('cybersecurity_intrusion_data.csv')
df = dataset
dataset_data = pd.DataFrame(dataset)

print("Shape:", df.shape)
dataset_data.head()


### 2.2 Teste de Duplicados

> Contagem de linhas/registos duplicados no dataset, podendo levar a uma limpeza prévia, para não prejudicar a análise realizada posteriormente!

In [ ]:
dataset.duplicated().sum()

### 2.3 Lidar com Valores Omissos

> Caso existam dados em falta, preencher os mesmos com `NaN` (Not a Number), atribuindo à variavel `data` para não influenciar o conteúdo original do `dataset` (como no caso `inplace=True`)

In [ ]:
data=dataset.fillna("No Data")

### 2.4 Pairplot

O Pairplot oferece uma visão multidimensional dos nossos dados, destacando as distribuições de cada **característica numérica** e as suas relações em pares, coloridas pela nossa variável alvo, `attack_detected`. Esta análise ajuda-nos a identificar padrões e potenciais preditores de ataques.

<br>

#### 2.4.1 Análise Gráficos da Diagonal (Distribuição de Variáveis Numéricas Individuais)

Os gráficos na diagonal (histogramas ou KDEs) mostram a distribuição de cada feature numérica, com as cores a diferenciar sessões sem ataque (azul, `attack_detected=0`) de sessões com ataque (laranja, `attack_detected=1`):

*   **`network_packet_size`**: A distribuição dos tamanhos dos pacotes de rede parece bastante sobreposta para ambas as classes. Isto sugere que o tamanho do pacote por si só pode não ser um forte indicador isolado de ataque, embora padrões subtis possam existir.

* **`login_attempts`**: A distribuição do número de tentativas de login para sessões de ataque (`attack_detected=1`) pode estar ligeiramente deslocada para valores mais altos, indicando que ataques podem envolver mais tentativas de login. Contudo, ainda pode haver uma sobreposição significativa com as sessões normais.

* **`session_duration`**: As durações das sessões também podem mostrar distribuições sobrepostas. Picos ou caudas distintas para a classe de ataque poderiam indicar durações atípicas (muito curtas para ataques rápidos, ou muito longas para infiltrações lentas) associadas a certos tipos de intrusões.

* **`ip_reputation_score`**: É provável que a distribuição para `attack_detected=1` (laranja) esteja mais concentrada em pontuações de reputação mais baixas, enquanto a distribuição para `attack_detected=0` (azul) tende a ter pontuações mais altas. IPs com má reputação são frequentemente usados em ataques.

* **`failed_logins`**: A distribuição para sessões de ataque (`attack_detected=1`) espera-se que mostre uma frequência mais elevada de `failed_logins` ou uma extensão para valores mais altos. Tentativas de login falhadas são um sinal comum de atividades maliciosas.

* **`unusual_time_access`**: Sendo binária (0 ou 1), a distribuição para `attack_detected=1` pode ter uma barra mais proeminente no valor `1`, sugerindo que ataques ocorrem mais frequentemente em horários incomuns.

<br>

#### 2.4.2 Análise Gráficos Fora da Diagonal (Relação entre Pares de Variáveis Numéricas)

Os gráficos de dispersão fora da diagonal revelam como dois pares de variáveis numéricas se relacionam, com a cor a indicar o estado do `attack_detected`:

* **`failed_logins` vs. `ip_reputation_score`**: Este é um dos gráficos mais importantes. Devemos procurar por um **agrupamento claro dos pontos laranja (`attack_detected=1`) na área superior esquerda**, ou seja, com um número elevado de `failed_logins` e uma baixa `ip_reputation_score`. Se for evidente, esta combinação é um forte indicador de ataque.

* **`unusual_time_access` vs. `failed_logins`**: Se `unusual_time_access` for 1 (acesso em horário incomum), e o número de `failed_logins` for alto, é provável que vejamos uma maior densidade de pontos laranja, sugerindo que estas condições combinadas são preditivas de ataques.

* **`ip_reputation_score` vs. `session_duration` / `network_packet_size`**: Podemos investigar se sessões de IPs com má reputação (baixa `ip_reputation_score`) mostram padrões distintos na sua duração ou no tamanho dos pacotes (e.g., sessões muito curtas para scans rápidos, ou muito longas para infiltração lenta), especialmente para os pontos laranja.

<br>

### 2.4.3 Conclusão:

A combinação de `failed_logins`, `ip_reputation_score`, e `unusual_time_access` parece ser promissora para diferenciar os ataques.

<br>

*(Nota: Variáveis categóricas como `protocol_type`, `encryption_used` e `browser_type` não são diretamente plotadas nos eixos do `pairplot` padrão, mas a sua influência é visível através da coloração (`hue`) quando a variável alvo é categórica. Para analisar essas variáveis individualmente ou em relação às numéricas, seriam necessários gráficos específicos para categorias, como countplots ou boxplots.)*

In [ ]:
#1 Pairplot
sns.pairplot(dataset_data, hue='attack_detected')
plt.show()

Após a visualização inicial com o Pairplot, removemos a coluna `session_id` do dataset pois é apenas um identificador e não uma feature útil para o modelo. Depois convertemos as colunas com tipo `'object'` para o tipo `'category'`, otimizando o uso de memória e preparando-as para codificação futura.
<br>

De seguida, usamos `dataset.dtypes` para verificação do tipo de cada parametro!

In [ ]:
dataset=dataset.drop(["session_id"],axis=1)

In [ ]:
for col in dataset.select_dtypes(include='object').columns:
    dataset[col] = dataset[col].astype('category')

In [ ]:
dataset.dtypes

Converte as colunas categóricas (não numéricas) para uma representação numérica. Utilizamos o `LabelEncoder` da Scikit-learn para atribuir um número inteiro único a cada categoria, tornando os dados compreensíveis para os algoritmos de modelagem.

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# encode all columns that are not of type integer or float (i.e., categorical columns)
for col in dataset.select_dtypes(exclude=['int64', 'float64']).columns:
    dataset[col] = label_encoder.fit_transform(dataset[col])

O One-Hot Encoding resolve este problema criando novas colunas binárias (0 ou 1) para cada categoria única em uma variável. Por exemplo, se tivermos a variável `protocol_type` com categorias 'TCP', 'UDP' e 'ICMP', o One-Hot Encoding criaria três novas colunas: `protocol_type_TCP`, `protocol_type_UDP` e `protocol_type_ICMP`. Para uma observação 'TCP', a coluna `protocol_type_TCP` teria o valor 1 e as outras 0. Isso garante que o modelo não interprete qualquer relação ordinal entre as categorias, tratando-as como entidades independentes.

Nesta implementação, utilizamos `pd.get_dummies()` que, por padrão, já lida com a remoção da primeira categoria (`drop_first=True`) para evitar a multicolinearidade, ou seja, a redundância de informação entre as novas colunas criadas.

In [ ]:
# One-Hot Encoding para todas as colunas categóricas
dataset = pd.get_dummies(
    dataset,
    columns=dataset.select_dtypes(exclude=['int64','float64']).columns,
    drop_first=True
)

### 2.5 Heatmap

O heatmap (matriz de correlação) visualiza a relação linear entre todas as variáveis numéricas do conjunto de dados, incluindo aquelas que foram codificadas a partir de categorias (`protocol_type`, `encryption_used`, `browser_type`). Os valores variam de -1 a 1, onde:

- **1** indica uma correlação positiva perfeita.
- **-1** indica uma correlação negativa perfeita.
- **0** indica nenhuma correlação linear.

<br>

#### Análise dos Resultados:

1.  **`attack_detected` (Variável Alvo):**
    *   **Correlações Positivas Fortes:** Observamos que `failed_logins` e `login_attempts` têm as correlações mais fortes e positivas com `attack_detected`. Isso sugere que um aumento no número de tentativas de login falhadas e nas tentativas de login está diretamente associado a uma maior probabilidade de deteção de ataque.
    *   **Correlações Negativas Fortes:** A variável `ip_reputation_score` apresenta uma correlação negativa notável com `attack_detected`. Isso implica que IPs com menor reputação (valores mais baixos) estão fortemente associados a ataques, o que é um resultado esperado.
    *   **Outras Correlações:** `unusual_time_access` também mostra uma correlação positiva, indicando que acessos fora do horário padrão contribuem para a deteção de ataques. `network_packet_size` e `session_duration` parecem ter correlações mais fracas, sugerindo que, isoladamente, podem não ser os preditores mais fortes.

2.  **Correlações entre Features (Multicolinearidade):**
    *   É importante observar também as correlações entre as próprias features. Correlações muito altas entre duas features preditoras podem indicar multicolinearidade, o que pode afetar a estabilidade de alguns modelos, mas geralmente não é um problema para modelos baseados em árvores como as Árvores de Decisão e Random Forests.

#### Nota:

O heatmap confirma que `failed_logins`, `ip_reputation_score`, `login_attempts` e `unusual_time_access` são as variáveis mais influentes na previsão de `attack_detected`.

In [ ]:
# Show the correlation Matrix
sns.heatmap(dataset.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

### 2.5.1 Boxplot

Os boxplots mostram a distribuição de cada variável numérica (`network_packet_size`, `login_attempts`, `session_duration`, `ip_reputation_score`, `failed_logins`) em relação à variável alvo `attack_detected` (0 para não detetado, 1 para detetado). Esta visualização é excelente para identificar diferenças nas medianas, dispersão e presença de *outliers* entre as duas classes.

#### Interpretação dos Boxplots:

*   **`network_packet_size`**: Se a mediana e a dispersão dos tamanhos de pacotes forem semelhantes para ambas as classes (0 e 1), esta variável pode não ser um forte preditor isolado de ataques. No entanto, se houver uma ligeira diferença ou *outliers* notáveis numa das classes, isso pode indicar padrões subtis.

*   **`login_attempts`**: Espera-se que as sessões com `attack_detected=1` apresentem uma mediana e/ou uma dispersão de `login_attempts` maior. Isto porque os ataques, como tentativas de força bruta, geralmente envolvem um número elevado de tentativas de login.

*   **`session_duration`**: A duração da sessão pode ser um indicador complexo. Ataques rápidos podem ter durações curtas, enquanto ataques de persistência podem ter durações longas. Observaremos se as distribuições para `attack_detected=1` mostram uma concentração em durações muito curtas ou muito longas, ou se são significativamente diferentes da classe `attack_detected=0`.

*   **`ip_reputation_score`**: Esta é uma variável crucial. É altamente provável que o boxplot para `attack_detected=1` esteja deslocado para valores mais baixos de `ip_reputation_score`, indicando que IPs com má reputação estão mais associados a ataques. A dispersão também pode ser menor, sugerindo que ataques tendem a vir de IPs com pontuações consistentemente baixas.

*   **`failed_logins`**: Semelhante a `login_attempts`, espera-se que as sessões com `attack_detected=1` mostrem um número significativamente maior de `failed_logins`. Uma mediana mais alta e/ou uma maior dispersão para esta classe seria um forte indício de atividade maliciosa.

#### Nota:

Variáveis com distribuições claramente separadas entre as duas classes (como provavelmente `failed_logins` e `ip_reputation_score`) serão os preditores mais fortes para os modelos.

In [ ]:
# Boxplots para variáveis numéricas face a 'attack_detected'

# Colunas numéricas adequadas para boxplots
numerical_cols = [
    'network_packet_size',
    'login_attempts',
    'session_duration',
    'ip_reputation_score',
    'failed_logins'
]

# Layout para os subplots
plt.figure(figsize=(18, 12)) # Tamanho geral da figura

for i, col in enumerate(numerical_cols):
    plt.subplot(2, 3, i + 1) # 2 linhas e 3 colunas
    sns.boxplot(x='attack_detected', y=col, data=dataset)
    plt.title(f'Distribuição de {col} por Ataque Detetado', fontsize=12)
    plt.xlabel('Ataque Detetado (0=Não, 1=Sim)', fontsize=10)
    plt.ylabel(col, fontsize=10)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)

plt.tight_layout() # Auto adjust -> parâmetros do subplot para que caibam na área da figura
plt.show()

### 2.5.2 Histograma de Tamanho de Pacotes de Rede por Deteção de Ataque

Este histograma visualiza a distribuição do `network_packet_size` (tamanho dos pacotes de rede) para sessões com e sem ataque, utilizando a variável `attack_detected` para diferenciação. Permite-nos observar a frequência de diferentes tamanhos de pacotes em ambas as classes.

#### Interpretação:

*   Ao analisar a sobreposição ou separação das distribuições para `attack_detected=0` (sem ataque) e `attack_detected=1` (com ataque), podemos verificar se o tamanho dos pacotes de rede é um indicador forte ou fraco de um ataque.
*   Se as distribuições forem muito semelhantes, o `network_packet_size` pode não ser, por si só, um preditor distintivo. No entanto, diferenças subtis em picos ou dispersão podem ainda oferecer alguma informação útil para o modelo.

Este gráfico complementa a análise dos boxplots, fornecendo uma visão mais detalhada da frequência de cada valor ou intervalo de valores do tamanho dos pacotes de rede.

In [ ]:
# Histogram of Network Packet Size by Attack Detected
for category in dataset_data['attack_detected'].unique():
	subset = dataset_data[dataset_data['attack_detected'] == category]
	sns.histplot(subset['network_packet_size'], label=f'Attack: {category}', kde=False)

plt.xlabel('Network Packet Size')
plt.ylabel('Frequency')
plt.title('Distribution of Network Packet Size by Attack Detected')
plt.legend()
plt.show()

# 2.6 Hold-Out

### 2.6.1 Model building

In [ ]:
x = dataset.drop(columns=['attack_detected'],axis=1)
y = dataset['attack_detected']

#Split the data
X_train , X_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=42,stratify=y)

### 2.6.2 Decision tree - Model building

Nesta secção, vamos construir o nosso primeiro modelo de Árvore de Decisão e tentar visualizá-lo para entender as regras que ele aprendeu.

<br>

1.  **Instanciação do Modelo de Árvore de Decisão**:
    *   **`dt = DecisionTreeClassifier(random_state=RANDOM_STATE)`**:
        *   Aqui, criamos uma instância do `DecisionTreeClassifier`. Esta é a classe em `sklearn.tree` que implementa o algoritmo da Árvore de Decisão. O parâmetro `random_state` é usado para garantir a reprodutibilidade dos resultados, ou seja, se você executar o código várias vezes com o mesmo `random_state`, a árvore gerada será sempre a mesma.

2.  **Treino do Modelo**:
    *   **`dt.fit(X_train, y_train)`**:
        *   Este passo é onde o modelo "aprende". O algoritmo da Árvore de Decisão analisa os dados de treino (`X_train`) e os respetivos rótulos (`y_train`) para construir a estrutura da árvore, identificando as melhores divisões (splits) em cada nó para otimizar a classificação da variável alvo `attack_detected`.

3.  **Visualização da Árvore de Decisão**:
    *   **`plot_tree(dt, feature_names=X.columns, class_names=[str(c) for c in sorted(y.unique())], filled=True, rounded=True)`**:
        *   A função `plot_tree` da `sklearn.tree` é utilizada para desenhar graficamente a árvore de decisão que acabámos de treinar. Esta visualização é extremamente útil para inspecionar as regras que o modelo aprendeu.
        *   **`feature_names=X.columns`**: Atribui os nomes das colunas de `X` (as features) a cada nó da árvore, tornando mais fácil entender qual feature está a ser usada para dividir os dados.
        *   **`class_names=[str(c) for c in sorted(y.unique())]`**: Atribui os nomes das classes da variável alvo (0 e 1, que representam "sem ataque" e "com ataque") aos nós da árvore.
        *   **`filled=True`**: Colore os nós da árvore com base na classe predominante naquele nó, com a intensidade da cor indicando a proporção de amostras daquela classe.
        *   **`rounded=True`**: Arredonda as bordas dos retângulos dos nós para uma estética melhor.

<br>

#### Nota:
Para árvores de decisão sem restrições de profundidade, como esta, a visualização pode resultar numa imagem extremamente grande e difícil de interpretar, pois a árvore pode ter aprendido regras muito específicas e complexas (o que pode levar a overfitting). Em muitos casos, limita-se a profundidade da árvore (`max_depth`) para facilitar a visualização e promover a generalização do modelo.

In [ ]:
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt.fit(X_train, y_train)

plt.figure(figsize=(12,8))
try:
    # Use 'features.columns' instead of 'X.columns'
    plot_tree(dt, feature_names=x.columns, class_names=[str(c) for c in sorted(y.unique())], filled=True, rounded=True)
    plt.show()
except Exception as e:
    print('Tree plotting failed:', e)

In [ ]:
# Visualization of the tree model dt_classifier
# (max_depth=3)
dt_small = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_STATE)
dt_small.fit(X_train, y_train)

plt.figure(figsize=(12,6))
# Use 'features.columns' instead of 'X.columns'
plot_tree(dt_small, feature_names=x.columns, class_names=[str(c) for c in sorted(y.unique())], filled=True, rounded=True)

plt.title('Shallow Decision Tree (max_depth=3)')
plt.show()

### 2.6.3 Avaliação do Modelo Decision Tree (Conjunto de Teste)

Esta célula tem como objetivo avaliar o desempenho do modelo `DecisionTreeClassifier` treinado anteriormente, utilizando o conjunto de dados de teste (`X_test`, `y_test`). A avaliação é feita através de métricas de classificação padrão:

1.  **`y_pred = dt.predict(X_test)`**:
    *   O modelo de Árvore de Decisão (`dt`) é usado para fazer previsões (`y_pred`) sobre a classe (`attack_detected`) dos dados no conjunto de teste (`X_test`).

2.  **`accuracy = accuracy_score(y_test, y_pred)`**:
    *   Calcula a **precisão (accuracy)** do modelo, que é a proporção de previsões corretas (tanto ataques corretamente detetados quanto não-ataques corretamente identificados) em relação ao total de amostras no conjunto de teste. Uma alta precisão indica que o modelo acerta na maioria das suas previsões.

3.  **`print(classification_report(y_test, y_pred))`**:
    *   Gera um **relatório de classificação** detalhado, que inclui:
        *   **Precision (Precisão)**: Das amostras que o modelo previu como sendo de uma determinada classe, quantas realmente pertenciam a essa classe.
        *   **Recall (Sensibilidade/Cobertura)**: Das amostras que realmente pertenciam a uma determinada classe, quantas foram corretamente identificadas pelo modelo.
        *   **F1-Score**: A média harmónica da precisão e do recall, útil para avaliar o modelo quando há um desequilíbrio entre as classes.
        *   **Support (Suporte)**: O número de ocorrências reais de cada classe no conjunto de teste.
    *   Este relatório é crucial para entender o desempenho do modelo em cada classe (0 para não-ataque, 1 para ataque), especialmente em cenários onde uma classe pode ser mais importante ou menos frequente que a outra.

4.  **Matriz de Confusão (`confusion_matrix`)**:
    *   **`cm = confusion_matrix(y_test, y_pred)`**: Calcula a matriz de confusão, que é uma tabela que descreve o desempenho de um modelo de classificação em um conjunto de dados de teste, para o qual os valores verdadeiros são conhecidos.
    *   **`sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ...)`**: Visualiza a matriz de confusão utilizando um heatmap, facilitando a interpretação. Os elementos da matriz são:
        *   **True Positives (TP)**: Ataques corretamente previstos como ataques (canto inferior direito).
        *   **True Negatives (TN)**: Não-ataques corretamente previstos como não-ataques (canto superior esquerdo).
        *   **False Positives (FP)**: Não-ataques incorretamente previstos como ataques (erro Tipo I - alarme falso) (canto superior direito).
        *   **False Negatives (FN)**: Ataques incorretamente previstos como não-ataques (erro Tipo II - ataque não detetado) (canto inferior esquerdo).
    *   A matriz de confusão é fundamental para entender os tipos de erros que o modelo está a cometer, o que é especialmente importante em deteção de intrusões, onde FNs (não detetar um ataque real) podem ter consequências graves.

In [ ]:
# Assess the model
y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification Report
print(classification_report(y_test,y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Decision Tree - Confusion Matrix (Test Set)')
plt.show()

### 2.6.4 Random Forest - Model building

Nesta secção, vamos construir e avaliar o nosso primeiro modelo de `RandomForestClassifier` com os seus parâmetros padrão, utilizando o conjunto de dados de treino e testando o seu desempenho no conjunto de teste.

1.  **Instanciação do Modelo Random Forest**:
    *   **`rf_model = RandomForestClassifier(random_state=RANDOM_STATE)`**: Criamos uma instância do `RandomForestClassifier`. O `random_state` é utilizado para garantir que os resultados do modelo sejam consistentes e reproduzíveis sempre que o código for executado.

2.  **Treino do Modelo**:
    *   **`rf_model.fit(X_train, y_train)`**: Este é o passo onde o modelo "aprende" os padrões e relações nos dados. O algoritmo Random Forest constrói múltiplas árvores de decisão com base nos `X_train` (features) e `y_train` (variável alvo).

3.  **Realização de Previsões**:
    *   **`y_pred_rf = rf_model.predict(X_test)`**: Após o treino, o modelo é usado para prever a classe (`attack_detected`) para as amostras no conjunto de teste (`X_test`), dados que o modelo nunca viu durante o treino.

4.  **Avaliação do Desempenho**:
    *   **`accuracy_rf = accuracy_score(y_test, y_pred_rf)`**: Calcula a precisão (accuracy) do modelo no conjunto de teste, que é a proporção de previsões corretas.
    *   **`print(classification_report(y_test, y_pred_rf))`**: Gera um relatório detalhado com métricas como Precision, Recall, F1-Score e Support para cada classe (0 e 1). Este relatório é fundamental para entender o desempenho do modelo em identificar cada tipo de sessão (normal vs. ataque).
    *   **Matriz de Confusão (`confusion_matrix`)**: A matriz de confusão (`cm_rf`) é visualizada para mostrar o número de True Positives, True Negatives, False Positives e False Negatives, fornecendo uma visão clara dos tipos de erros que o modelo está a cometer.

<br>

#### Nota:
 Esta avaliação serve como uma **linha de base** para o desempenho do `RandomForestClassifier` antes de qualquer otimização ou ajuste de hiperparâmetros. Ela ajuda a entender o desempenho inicial do modelo com suas configurações padrão.

In [ ]:
# Instantiate a default Random Forest Classifier
rf_model = RandomForestClassifier(random_state=RANDOM_STATE)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Calculate Accuracy Score
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest - Test Accuracy: {accuracy_rf * 100:.2f}%")

# Print Classification Report
print("\nRandom Forest - Test Set Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Plot Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(6,4))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
plt.title('Random Forest - Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 2.7 Cross-validation

## 2.7.1 Decision Tree - Model instantiation

In [ ]:
# Prepare the data
feature_names = x.columns.tolist()
features = x

target = y
target_names = [str(c) for c in sorted(y.unique())]

print('Features:', feature_names, 'Classes:', target_names)

# Instantiate the model
cv_classifier = DecisionTreeClassifier(random_state=RANDOM_STATE)

### Decision Tree - Model assessment

In [ ]:
# Evaluate the model using cross validation
acc_score = cross_val_score(cv_classifier, features, target, cv=10)
print("CV Mean Accuracy: %0.3f (+/- %0.3f)" % (acc_score.mean(), acc_score.std()) )

f1_score = cross_val_score(cv_classifier, features, target, cv=10, scoring='f1_macro')
print("CV Mean F1: %0.3f (+/- %0.3f)" % (np.mean(f1_score), np.std(f1_score)) )

### Decision Tree - Model deployment

In [ ]:
# Build the model with the complete data
final_classifier = cv_classifier.fit(features, target)

In [ ]:
# Make predictions on new data
# X_new must have the same number of features as the training data (X)
# We'll create a sample X_new with all zeros for demonstration, matching the 13 features
X_new = pd.DataFrame(0, index=[0], columns=features.columns)

# X_new['network_packet_size'] = 500
# X_new['login_attempts'] = 2
# X_new['protocol_type_TCP'] = 1 # Assuming TCP
# X_new['failed_logins'] = 5
# X_new['ip_reputation_score'] = 0.9

prediction = final_classifier.predict(X_new)
print("Prediction:", prediction)

In [ ]:
# Model assessment: Precision Recall scores and Confusion matrix
print("Precision, Recall, Confusion matrix")
print(classification_report(target, final_classifier.predict(features), digits=3))

cm = confusion_matrix(target, final_classifier.predict(features))
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
plt.title('Final Classifier - Confusion Matrix (Training Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### Fixing Overfitting: Using `max_depth` in Decision Trees

To mitigate overfitting in a Decision Tree, we can limit its complexity. A common way is to set the `max_depth` hyperparameter, which restricts how deep the tree can grow. This prevents the model from memorizing the training data too specifically.

Let's train a new Decision Tree with a `max_depth` of, for example, 7, and compare its performance on both the training and test sets. We expect the training accuracy to be lower than 100%, but the test accuracy should be more representative and potentially higher than an unconstrained, overfit tree.

In [ ]:
# Instantiate a Decision Tree Classifier with a limited max_depth
dt_fixed = DecisionTreeClassifier(min_samples_leaf = 10, max_depth=7, random_state=RANDOM_STATE)

# Train the model on the training data
dt_fixed.fit(X_train, y_train)

# Evaluate on the training set
y_train_pred_fixed = dt_fixed.predict(X_train)
train_accuracy_fixed = accuracy_score(y_train, y_train_pred_fixed)
print(f"Fixed Decision Tree - Training Accuracy: {train_accuracy_fixed * 100:.2f}%")

# Evaluate on the test set
y_test_pred_fixed = dt_fixed.predict(X_test)
test_accuracy_fixed = accuracy_score(y_test, y_test_pred_fixed)
print(f"Fixed Decision Tree - Test Accuracy: {test_accuracy_fixed * 100:.2f}%")

# Print classification report for the test set
print("\nFixed Decision Tree - Test Set Classification Report:")
print(classification_report(y_test, y_test_pred_fixed))

# Plot Confusion Matrix for the test set
cm_fixed_dt = confusion_matrix(y_test, y_test_pred_fixed)
plt.figure(figsize=(6,4))
sns.heatmap(cm_fixed_dt, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
plt.title('Fixed Decision Tree (max_depth=7) - Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Random Forest - Model instatiation

In [ ]:
# Instantiate the Random Forest model for cross-validation
rf_cv_classifier = RandomForestClassifier(random_state=RANDOM_STATE)

### Radom Forest - Model assessment

In [ ]:
# Evaluate the model using cross validation for accuracy
rf_acc_score = cross_val_score(rf_cv_classifier, features, target, cv=10)
print("Random Forest CV Mean Accuracy: %0.3f (+/- %0.3f)" % (rf_acc_score.mean(), rf_acc_score.std()) )

# Evaluate the model using cross validation for F1 score
rf_f1_score = cross_val_score(rf_cv_classifier, features, target, cv=10, scoring='f1_macro')
print("Random Forest CV Mean F1: %0.3f (+/- %0.3f)" % (np.mean(rf_f1_score), np.std(rf_f1_score)) )

### Random Forest - Model deployment

In [ ]:
# Train the final Random Forest model on the complete data
final_rf_classifier = RandomForestClassifier(random_state=RANDOM_STATE)
final_rf_classifier.fit(features, target)

print("Final Random Forest model trained on full dataset.")

# Make predictions on new data using the final Random Forest model
prediction_rf = final_rf_classifier.predict(X_new)
print(f"Prediction for new data using Random Forest: {prediction_rf}")

In [ ]:
# Model assessment: Precision Recall scores and Confusion matrix (on training data)

print("\nRandom Forest - Training Set Precision, Recall, Confusion matrix:")
print(classification_report(target, final_rf_classifier.predict(features), digits=3))

cm_rf_deployment = confusion_matrix(target, final_rf_classifier.predict(features))
plt.figure(figsize=(6,4))
sns.heatmap(cm_rf_deployment, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
plt.title('Final Random Forest Classifier - Confusion Matrix (Training Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### 2.5.3 Otimização do Modelo (com `max_depth`)

Embora Random Forests sejam menos propensos a overfitting que árvores individuais, limitar a profundidade máxima das árvores constituintes (`max_depth`) pode melhorar a generalização e reduzir o tempo de treino, especialmente se as árvores padrão forem muito profundas.

Sendo assim, iremos treinar e avaliar um modelo Random Forest com `max_depth` limitado a 7.

In [ ]:
# Instantiate a Random Forest Classifier with a limited max_depth for its estimators
rf_fixed = RandomForestClassifier(max_depth=7, random_state=RANDOM_STATE)

# Train the model on the training data
rf_fixed.fit(X_train, y_train)

# Evaluate on the training set
y_train_pred_rf_fixed = rf_fixed.predict(X_train)
train_accuracy_rf_fixed = accuracy_score(y_train, y_train_pred_rf_fixed)
print(f"Fixed Random Forest - Training Accuracy: {train_accuracy_rf_fixed * 100:.2f}%")

# Evaluate on the test set
y_test_pred_rf_fixed = rf_fixed.predict(X_test)
test_accuracy_rf_fixed = accuracy_score(y_test, y_test_pred_rf_fixed)
print(f"Fixed Random Forest - Test Accuracy: {test_accuracy_rf_fixed * 100:.2f}%")

# Print classification report for the test set
print("\nFixed Random Forest - Test Set Classification Report:")
print(classification_report(y_test, y_test_pred_rf_fixed))

# Plot Confusion Matrix for the test set
cm_fixed_rf = confusion_matrix(y_test, y_test_pred_rf_fixed)
plt.figure(figsize=(6,4))
sns.heatmap(cm_fixed_rf, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
plt.title('Fixed Random Forest (max_depth=7) - Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

---
# 3. **CONCLUSÃO**

## 3.1 Geral
Esta secção resume as descobertas do projeto e descreve as direções futuras.

O projeto implementou e avaliou com sucesso modelos de Árvore de Decisão e Random Forest para deteção de intrusões em cibersegurança. Ambos os modelos demonstraram um bom desempenho, com o Random Forest a apresentar uma precisão superior e um F1-score no conjunto de teste, indicando a sua robustez na identificação de cenários de ataque e não ataque. A engenharia de características, particularmente a codificação de variáveis categóricas e a análise de correlação e distribuição, foi crucial na preparação dos dados para um treino eficaz do modelo. Preditoras-chave identificadas, como `failed_logins` e `ip_reputation_score`, revelaram-se altamente influentes na distinção de atividades maliciosas.

## 3.2 Desafios e soluções
**Desafios:**
- **Desequilíbrio de Dados:** Um potencial desequilíbrio entre instâncias de ataque e não ataque foi um desafio que exigiu métricas de avaliação cuidadosas (como o F1-score) além da simples precisão.
- **Overfitting em Árvores de Decisão:** Os modelos iniciais de Árvore de Decisão sem restrições de profundidade exibiram 100% de precisão no treino, mas menor precisão no teste, indicando overfitting.
- **Interpretabilidade vs. Desempenho:** Embora as Árvores de Decisão ofereçam alta interpretabilidade, os Random Forests proporcionaram um melhor desempenho geral, mas são menos diretos de interpretar diretamente.

**Soluções:**
- **Divisão Estratificada:** Usou-se `stratify=y` em `train_test_split` para manter a distribuição das classes nos conjuntos de treino e teste.
- **Parâmetro `max_depth`:** Limitou-se a `max_depth` das Árvores de Decisão para prevenir o overfitting, levando a uma melhor generalização em dados não vistos.
- **Métodos de Ensemble:** Empregou-se Random Forest para aproveitar o poder de múltiplas árvores, mitigando as fraquezas das árvores individuais e melhorando o poder preditivo geral.

## 3.3 Perspetivas Futuras
Trabalhos futuros poderiam envolver:
- **Otimização de Hiperparâmetros:** Otimização adicional dos hiperparâmetros da Árvore de Decisão e do Random Forest usando técnicas como GridSearchCV ou RandomizedSearchCV para potencialmente alcançar um desempenho ainda maior.
- **Engenharia de Características Avançada:** Exploração de características mais complexas ou termos de interação (e.g., combinando `login_attempts` com `unusual_time_access`).
- **Outros Modelos:** Investigação de outros algoritmos de machine learning, como Gradient Boosting Machines (XGBoost, LightGBM) ou redes neurais para comparação.
- **Deteção em Tempo Real:** Adaptação dos modelos para deteção de intrusões em tempo real, considerando dados de streaming e abordagens de aprendizagem incremental.

## 3.4 Em retrospetiva
O processo destacou a importância de uma análise exploratória de dados (EDA) aprofundada para entender as distribuições e correlações das características. A Árvore de Decisão inicial não restrita serviu como uma lição valiosa na identificação e abordagem do overfitting. O desempenho robusto do modelo Random Forest sublinha a eficácia da aprendizagem por conjunto em tarefas de classificação complexas, como a deteção de intrusões em cibersegurança. A clara interpretabilidade da função `plot_tree` para árvores rasas também foi benéfica para a compreensão das regras básicas de decisão.

<br>

Obrigado, Professor Joaquim :)